In [1]:
import os


In [2]:
os.chdir('../')

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class  Model_prepare_Config:
    root_dir:Path
    training_model:Path
    text_feature:Path
    params_image_size:list
    params_learning_rate:float
    params_weights:str
    params_include_top:bool
    
    


In [4]:
from Image2Recipie.constants import *
from Image2Recipie.utils.common import read_yaml,create_directories
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bisht\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH

    ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

    def get_prepare_model_config(self)->Model_prepare_Config:
        config=self.config
        create_directories([config.prepare_model.root_dir])
        prepare_model_config=Model_prepare_Config(
            root_dir=self.config.prepare_model.root_dir,
            training_model=self.config.prepare_model.training_model,
            text_feature=os.path.join(self.config.preprocessing.root_dir,'features.pkl'),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS

            
        )
        return prepare_model_config

In [6]:
import os
import urllib.request as request 
import gdown
from Image2Recipie import logger
import tensorflow as tf
from Image2Recipie.utils.common import save_object,get_maxlen,load_object
from tensorflow.keras.layers import Dense,Concatenate,LSTM,Bidirectional,Dropout,Input,GlobalAvgPool2D,Embedding,LayerNormalization,Bidirectional,BatchNormalization
import cv2 as cv
import pandas as pd
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [21]:
class PrepareModel:
    def __init__(self,config:Model_prepare_Config):
        self.config = config
        self.input_image=None 
    @staticmethod
    def save_model(path:Path,model:tf.keras.Model):
        model.save(path)
    
    def prepare_model(self):
        features=load_object(self.config.text_feature)
        print(features)
        image_input = Input(shape=(160, 160, 3))  # Example input shape for ResNet50
        text_input = Input(shape=(features['input_maxlen'],))  # Text input

        # Base model for image input (ResNet50)
        base_model = ResNet50(include_top=self.config.params_include_top, weights=self.config.params_weights, input_shape=self.config.params_image_size)
        base_model.trainable = False

        # Get the output from the base model
        x = base_model(image_input)  # Shape: (batch_size, height, width, channels)

        # Flatten the output from the base model
        x = GlobalAvgPool2D()(x)  # Shape: (batch_size, channels)
        x = Dense(128, activation='relu')(x)
        x=BatchNormalization()(x)
        x=Dropout(0.4)(x)
        x = Dense(64, activation='relu')(x)
        x=BatchNormalization()(x)
        x=Dropout(0.2)(x)
        x = Dense(32, activation='relu')(x)
        x=BatchNormalization()(x)
        # Text input processing
        masking = tf.keras.layers.Masking(mask_value=0)  # Set mask_value as scalar (commonly 0)
        encoder_inputs_masked = masking(text_input)

        # Embedding and LSTM layers for text input
        embed = Embedding(input_dim=features['input_vocal_size']+1, output_dim=128, input_length=features['input_maxlen'])(encoder_inputs_masked)
        lstm2 = Bidirectional(LSTM(64, return_sequences=True))(embed)
        layer_norm=LayerNormalization()(lstm2)
        lstm3 = Bidirectional(LSTM(32, return_sequences=False))(layer_norm)
        layer_norm1=LayerNormalization()(lstm3)
        concatenate=Concatenate()([x,layer_norm1])
        # Concatenate image and text features

        encoder_state_h = Dense(32)(concatenate)
        encoder_state_c = Dense(32)(concatenate)
        encoder_states = [encoder_state_h, encoder_state_c]

        # Decoder input
        decoder_input = Input(shape=(None,), name='decoder_inputs')  # Changed shape

        # Decoder embedding
        decoder_embedding = Embedding(input_dim=features['output_vocab_size'] + 1,  # Add +1 for padding token
            output_dim=32,
                                    mask_zero=True)(decoder_input)

        # Decoder LSTM
        decoder_lstm = LSTM(32, return_sequences=True, return_state=True, name='decoder_lstm')
        decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

        # Final dense layer for decoder outputs
        decoder_dense = Dense(features['output_vocab_size']+1, activation='softmax', name='decoder_dense')  # Changed output dimension
        decoder_outputs = decoder_dense(decoder_outputs)

        # Define the final model
        model = Model(inputs=[image_input, text_input, decoder_input], outputs=decoder_outputs)
        adam=Adam(learning_rate=self.config.params_learning_rate)
        model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
        model.summary()
        self.save_model(path=self.config.training_model,model=model)

In [22]:
try:
    config=ConfigurationManager()
    model_prepare_config=config.get_prepare_model_config()
    data_preprocessing=PrepareModel(config=model_prepare_config)
    data_preprocessing.prepare_model()
    
except Exception as e:
    raise e

[2024-11-03 23:08:11,888:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-11-03 23:08:11,895:INFO:common:yaml file: params.yaml loaded successfully]
[2024-11-03 23:08:11,912:INFO:common:created directory at: artifacts/prepare_model]


{'max_outputtoken': 44, 'output_vocab_size': 992, 'input_maxlen': 80, 'input_vocal_size': 2391}


c:\Users\bisht\OneDrive\Desktop\image-to-recipie\venv\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'embedding_12' (of type Embedding) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_13      │ (None, 160, 160,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50            │ (None, 5, 5,      │ 23,587,712 │ input_layer_13[0… │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 2048)      │          0 │ resnet50[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_30 (Dense)    │ (None, 128)       │    262,272 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_30[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_14      │ (None, 80)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_12          │ (None, 128)       │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking_6 (Masking) │ (None, 80)        │          0 │ input_layer_14[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_31 (Dense)    │ (None, 64)        │      8,256 │ dropout_12[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_12        │ (None, 80, 128)   │    306,176 │ masking_6[0][0]   │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_31[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_12    │ (None, 80, 128)   │     98,816 │ embedding_12[0][… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_13          │ (None, 64)        │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 80, 128)   │        256 │ bidirectional_12… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_32 (Dense)    │ (None, 32)        │      2,080 │ dropout_13[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_13    │ (None, 64)        │     41,216 │ layer_normalizat… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32)        │        128 │ dense_32[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 64)        │        128 │ bidirectional_13… │
│ (LayerNormalizatio… │                   │            │                 

 Total params: 24,386,881 (93.03 MB)

 Trainable params: 798,721 (3.05 MB)

 Non-trainable params: 23,588,160 (89.98 MB)